In [1]:
#Imports
import os
import xml.etree.ElementTree as ET
import pandas as pd
from datetime import datetime
import pickle
import tqdm
import line_profiler
from operator import itemgetter
from sklearn.preprocessing import OneHotEncoder
import math

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
def track_id_gen(dist, track):
    if track==None:
        return "NA"
    return track[0:4]+"-"+str(int(dist))

In [3]:
os.getcwd()
os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")

In [4]:
race_ft_details = pd.DataFrame(pickle.load(open('race_details_ft.npy','rb')), columns=['@id', 'RaceNum', 'RaceName','RaceTime', 'Distance', 'RaceGrade', 'Track', 'date'])
race_ft_details['RaceId'] = race_ft_details['@id'] 
dog_ft_details = pd.DataFrame(pickle.load(open('dog_details_ft.npy','rb')))
dog_ft_details['DogId'] =   dog_ft_details['@id']

In [5]:
##Convert Margins:
dog_ft_details['SplitTimes']= dog_ft_details.SplitMargin.astype(float)
dog_ft_details['minMargin'] = dog_ft_details.groupby('RaceId')['SplitTimes'].transform('min')
dog_ft_details['SplitMargin'] = dog_ft_details.SplitTimes-dog_ft_details.minMargin
dog_ft_details['Margin1'] = dog_ft_details['Margin1'].fillna(0)
dog_ft_details['StartPrice'] = dog_ft_details['StartPrice'].fillna(0)
dog_ft_details['DogName'] = dog_ft_details['DogName'].apply(lambda x: x.replace("'", "").replace(".", "").replace("Res", "").strip() if x else x)

In [7]:
dog_ft_details

,@id,Place,DogName,Box,Rug,Weight,StartPrice,Handicap,Margin1,Margin2,...,Comments,SplitMargin,RunTime,Prizemoney,RaceId,TrainerId,TrainerName,DogId,SplitTimes,minMargin
0,314367490,1,SUPER BIG,6,6,32.0,$4.70,None,5.91,None,...,None,0.00,19.24,None,501007024,55811,J Boreland,314367490,6.58,6.58
1,296096022,2,RIO DIXIE,1,1,32.2,$2.80F,None,5.91,5.91,...,None,0.12,19.66,None,501007024,55811,J Boreland,296096022,6.70,6.58
2,304203706,3,COCO JUMBO,8,8,29.9,$3.70,None,6.99,1.07,...,None,0.17,19.73,None,501007024,22077,A Terry,304203706,6.75,6.58
3,291188715,4,BEARING GOLD,4,4,29.2,$32.00,None,8.46,1.47,...,None,0.23,19.84,None,501007024,103190,C Watson,291188715,6.81,6.58
4,317789815,5,ROLLIN IN,3,3,28.2,$15.80,None,11.31,2.86,...,None,0.23,20.04,None,501007024,141797,M Craig,317789815,6.81,6.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1264027,549390093,6,DIRTY HARRY,2,2,30.1,$7.00,None,7.50,1.57,...,Mcgo8sow,0.50,17.99,0.00,846624262,24348,V Cook,549390093,7.33,6.83
1264028,550748030,7,GILDED GOAT,5,5,29.7,$47.20,None,7.75,0.43,...,Mc4,0.28,18.02,0.00,846624262,102994,S Beard,550748030,7.11,6.83
1264029,338855457,8,HES MY MAC,3,3,31.0,$39.50,None,9.75,1.86,...,Mcgxccxcc7,0.47,18.15,0.00,846624262,58604,K McLennan,338855457,7.30,6.83
1264030,444688105,R,TRUSS,None,9,None,0,None,0,None,...,None,NaN,None,None,846624262,69407,P Worthington,444688105,NaN,6.83


In [8]:
full_details = pd.merge(race_ft_details,dog_ft_details, 'right', on='RaceId')

In [9]:
len(full_details)

1264032

In [10]:
full_details.to_csv('full_race_details_ft.csv')

In [11]:

resultsdf = full_details
print(resultsdf.head())
split_distances = pd.read_csv("split_dist_ft.csv")
resultsdf["dist"] = resultsdf["Distance"].astype(str).str[:-1].astype(float)

resultsdf.run_time = resultsdf.RunTime.astype(float)
resultsdf["track_id"] = resultsdf.apply(lambda s: track_id_gen(s["dist"],s["Track"]), axis=1)
resultsdf_merged = pd.merge(resultsdf, split_distances, on=["track_id", "Track"])

       @id_x RaceNum                     RaceName RaceTime Distance  \
0  501007024       1  FAST FREDDIE'S TYRE SERVICE  11:00AM     350m   
1  501007024       1  FAST FREDDIE'S TYRE SERVICE  11:00AM     350m   
2  501007024       1  FAST FREDDIE'S TYRE SERVICE  11:00AM     350m   
3  501007024       1  FAST FREDDIE'S TYRE SERVICE  11:00AM     350m   
4  501007024       1  FAST FREDDIE'S TYRE SERVICE  11:00AM     350m   

   RaceGrade        Track       date     RaceId      @id_y  ... Checks  \
0  Mixed 6/7  Healesville  01 Dec 19  501007024  314367490  ...      0   
1  Mixed 6/7  Healesville  01 Dec 19  501007024  296096022  ...      0   
2  Mixed 6/7  Healesville  01 Dec 19  501007024  304203706  ...      1   
3  Mixed 6/7  Healesville  01 Dec 19  501007024  291188715  ...      2   
4  Mixed 6/7  Healesville  01 Dec 19  501007024  317789815  ...      8   

  Comments SplitMargin RunTime Prizemoney TrainerId TrainerName      DogId  \
0     None        0.00   19.24       None     5581

C:\Users\Nick\AppData\Local\Temp\ipykernel_6848\282841814.py:6: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  resultsdf.run_time = resultsdf.RunTime.astype(float)


In [12]:
betfair_df = pickle.load(open('df-betfairSP.npy', 'rb'))
betfair_df['dateF'] = pd.to_datetime(betfair_df.EVENT_DT, dayfirst=True).dt.date
betfair_df['dog_name'] = betfair_df.dog.str[1:].str.upper()
resultsdf['dateF']= pd.to_datetime(resultsdf.date).dt.date

In [13]:
betfair_df

,EVENT_ID,EVENT_DT,SELECTION_ID,BSP,dog,MENU_HINT,dateF,dog_name
2,138449114,31-12-2017 08:08,9117762,40.000000,Magic Moonlight,AUS / Canb (AUS) 31st Dec,2017-12-31,MAGIC MOONLIGHT
3,138449293,31-12-2017 09:34,10880108,7.000000,Better Size,AUS / APrk (AUS) 31st Dec,2017-12-31,BETTER SIZE
5,138449295,31-12-2017 09:54,11342999,6.800000,Midnight Dasher,AUS / APrk (AUS) 31st Dec,2017-12-31,MIDNIGHT DASHER
34,138448884,31-12-2017 02:45,12190032,9.600000,Redda,AUS / MGam (AUS) 31st Dec,2017-12-31,REDDA
37,138424801,31-12-2017 11:05,12600557,21.000000,Navajo Warrior,AUS / Darw (AUS) 31st Dec,2017-12-31,NAVAJO WARRIOR
...,...,...,...,...,...,...,...,...
3273448,208087087,30-12-2022 05:57,42361028,15.183486,Ibiza,AUS / The Gardens (AUS) 30th Dec,2022-12-30,IBIZA
3273449,208087093,30-12-2022 06:37,42162715,56.958384,Sugar Jack,AUS / The Gardens (AUS) 30th Dec,2022-12-30,SUGAR JACK
3273450,208087096,30-12-2022 06:57,51312699,3.541551,Wilsons Pick,AUS / The Gardens (AUS) 30th Dec,2022-12-30,WILSONS PICK
3273451,208087102,30-12-2022 07:39,28490220,3.350000,Bat Mobile,AUS / The Gardens (AUS) 30th Dec,2022-12-30,BAT MOBILE


In [19]:
resultsdf_merged.Track.value_counts()


Christchurch (NZ)    48081
Albion Park          46997
Cannington           44964
Mandurah             44499
Ipswich              40252
                     ...  
Potts Park             749
Taranaki (NZ)          118
Tokoroa (NZ)            92
Coonabarabran           58
Canberra                 1
Name: Track, Length: 73, dtype: int64

In [20]:
resultsdf.Track.value_counts()

Christchurch (NZ)    48081
Albion Park          46997
Cannington           44964
Mandurah             44499
Ipswich              40252
                     ...  
Potts Park             749
Taranaki (NZ)          118
Tokoroa (NZ)            92
Coonabarabran           58
Canberra                 1
Name: Track, Length: 73, dtype: int64

In [21]:
# pickle Load results
# resultsdf = pd.DataFrame(pickle.load(open('race_details_ft.npy','rb')))
# resultsdf.head()
resultsdf = full_details
ohe = OneHotEncoder(dtype=int,sparse=False)
transformed = ohe.fit_transform(resultsdf[['Track']])
#enc = OneHotEncoder( categories=tracks)
print(resultsdf.shape)
print(len(transformed[0]))

resultsdf["tracksOnehot"] = transformed.tolist()


split_distances = pd.read_csv("split_dist_ft.csv")
resultsdf["dist"] = resultsdf["Distance"].astype(str).str[:-1].astype(float)
resultsdf = resultsdf[resultsdf['RunTime'].notnull()]
resultsdf['run_time'] = pd.to_numeric(resultsdf['RunTime'])
resultsdf['split_margins'] = resultsdf.SplitMargin.astype(float)

resultsdf['place'] = resultsdf.Place.str[0].astype(float)
resultsdf["track_id"] = resultsdf.apply(
    lambda s: track_id_gen(s["dist"], s["Track"]), axis=1
)

print(len(resultsdf))
resultsdf_merged = pd.merge(resultsdf, split_distances, on=["track_id", "Track"], how='left')
print("here", len(resultsdf_merged))
resultsdf_merged = resultsdf_merged[resultsdf_merged['RunTime'].notnull()]
resultsdf_merged["dateF1"] = pd.to_datetime(resultsdf_merged["date"], format="%d %b %y")
resultsdf_merged["StartPrice"] = pd.to_numeric(resultsdf_merged['StartPrice'].str[1:-1])
resultsdf_merged["speed"] = pd.to_numeric(resultsdf_merged["RunTime"]) / pd.to_numeric(
    resultsdf_merged["dist_x"]
)
resultsdf_merged['split_dist_estim'] = resultsdf_merged['split_dist_estim'].fillna(100)
resultsdf_merged["split_speed"] = pd.to_numeric(
    resultsdf_merged["SplitTimes"]
) / pd.to_numeric(resultsdf_merged["split_dist_estim"])

resultsdf_merged["box"] = pd.to_numeric(resultsdf_merged["Box"])
resultsdf_merged["margin"] = pd.to_numeric(resultsdf_merged["Margin1"])
resultsdf_merged["weigth"] = pd.to_numeric(resultsdf_merged["Weight"])
resultsdf_merged['split_margins'] = pd.to_numeric(resultsdf_merged.SplitMargin)
resultsdf_merged["dateF"] = pd.to_datetime(resultsdf_merged["date"], format="%d %b %y").dt.date
resultsdf_merged['dog_name'] = resultsdf_merged['DogName']

resultsdf_merged = pd.merge(
    resultsdf_merged, betfair_df, how="left", on=["dateF", "dog_name"]
)

form = resultsdf_merged.sort_values("dateF", ascending=False)
form.loc[form['place']==1, 'margin']=0
with open("all-results-db.npy", "wb") as fp:   #Pickling
    
    pickle.dump(form, fp)
b
    
print(form.head())

form = form.sort_values(['dateF'])
race_forms = form.groupby(["RaceId"])
dog_forms = form.groupby(["DogId"])
print(race_forms)
race_input_layer = []
race_classes = []
full_details_new = []
dog_stats_df = []
resultsdf_merged.head(50)
n = 0
for race, j in tqdm.tqdm(race_forms):
    try:
        race_date = pd.to_datetime(j["dateF"].iloc[0])
        dogs = j['DogId']
        race_id = race
        race_grade = j.RaceGrade.iloc[0]
        race_dist = j.dist_x.iloc[0]
        trackOHE = j.tracksOnehot.iloc[0]
        track_name = j.Track.iloc[0]
        dog_info = []
        num_features = 16
        

        #Starts off with blank outfits of what we need to fill, are then filled using box pos as indexer
        blank_stats = [-1]*num_features
        blank_race = [blank_stats]*8
        blank_classes = [8]*8
        blank_margins = [80]*8
        blank_sp = [0]*8
        blank_bsp = [0]*8
        blank_dogname = ["blank"]*8
        if len(dogs)>8:
            continue
        i = 0
        for dog in dogs:
            
            #print(f"dog id = {dog}")
            # if i ==1:
            #     print(f"{race_date=}")
            
            prev_form = dog_forms.get_group(dog)
            #print(f"{len(prev_form)=}")
            prev_form = prev_form[(prev_form["dateF"] < race_date)]
            #print(f"After date cut: {len(prev_form)=}")
            prev_races = len(prev_form)


            dog_speed_avg = prev_form.speed.mean()
            dog_speed_sd = prev_form.speed.std()
            dog_speed_max = prev_form.speed.max()
            dog_split_avg = prev_form.split_speed.mean()
            dog_split_sd = prev_form.split_speed.std()
            dog_split_max = prev_form.split_speed.max()
            dog_split_mg_avg = prev_form.split_margins.mean()
            dog_margin_mean = prev_form.margin.mean()
            dog_margin_sd = prev_form.margin.std()
            if prev_races>0:
                dog_margin_last = prev_form.margin.iloc[0]
                dog_speed_last = prev_form.speed.iloc[0]
                dog_race_count = prev_form.shape[0]
                try:
                    dog_wins = len(prev_form[prev_form["place"] == 1])
                    dog_places = len(prev_form[prev_form["place"] <= 3])
                except Exception as e:
                    print("no win")
            else:
                dog_margin_last = -1
                dog_speed_last = -1
                dog_race_count = 0
                dog_wins = 0
                dog_places = 0
            # print(dog_margin_last)
            #print()

            dog_box = j.box.iloc[i]
            dog_weight = j.weigth.iloc[i] # weight not in forms for prediction going to use last weight
            stats = [
                    dog_box,
                    dog_speed_avg,
                    dog_speed_sd,
                    dog_speed_max,
                    dog_split_avg,
                    dog_split_sd,
                    dog_split_max,
                    dog_split_mg_avg,
                    dog_margin_mean,
                    dog_margin_sd,
                    dog_race_count,
                    dog_wins,
                    dog_places,
                    dog_weight,
                    dog_speed_last,
                    dog_margin_last
                    ]
            #print(stats)
            stats = [-1 if math.isnan(x) else x for x in stats]
            dog_stats = (
                dog,
                stats,
                j.place.iloc[i],
                j.StartPrice.iloc[i],
                j.margin.iloc[i],
                j.BSP.iloc[i],
            )

            #Fill in the blank outputs
            idx = dog_box-1
            blank_race[idx] = stats
            blank_classes[idx] = j.place.iloc[i]
            blank_bsp[idx] = j.BSP.iloc[i]
            blank_margins[idx] = j.margin.iloc[i]
            blank_sp[idx] = j.StartPrice.iloc[i]
            blank_dogname[idx] = j.dog_name.iloc[i]




            dog_info.append(dog_stats)
            dog_stats_df.append([dog,
                    j.dog_name.iloc[i],
                    race_id,
                    race_grade,
                    race_date,
                    trackOHE,
                    track_name,
                    race_dist,
                    stats,
                    j.place.iloc[i],
                    j.StartPrice.iloc[i],
                    j.margin.iloc[i],
                    j.BSP.iloc[i],
                    j.RunTime.iloc[i]])
            #print(f"{dog_stats_df=}")
            i = i + 1
        
        

        dog_info.sort(key=itemgetter(0))
        # dog_stats_df.append(dog_info)
        input_layer = (
            trackOHE
            + [race_dist]
            + [item for sublist in blank_race for item in sublist]
        )
        
        # if len(dogs) > 8:
        #     print(f"{race_id=}")
        #     print(f"{race_date=}")
        #     print(f"{j.track_x.iloc[1]=}")
        classes = blank_classes
        prices = blank_sp
        margins = blank_margins
        betfairSP = blank_bsp
        dogname = blank_dogname
        race_input_layer.append(input_layer)
        race_classes.append(classes)
        full_details_new.append(
            (race_id, input_layer, classes, prices,  margins, betfairSP, dogname)
        )

        
    except Exception as e:
        print(f"{e=}")
    # n += 1
    # if n >10:
    #     break

(1264032, 34)
74


C:\Users\Nick\AppData\Local\Temp\ipykernel_6848\2047770890.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultsdf['run_time'] = pd.to_numeric(resultsdf['RunTime'])
C:\Users\Nick\AppData\Local\Temp\ipykernel_6848\2047770890.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultsdf['split_margins'] = resultsdf.SplitMargin.astype(float)
C:\Users\Nick\AppData\Local\Temp\ipykernel_6848\2047770890.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try u

1063794
here 1063794


NameError: name 'b' is not defined

full_details

In [ ]:
stop 

testdf = pd.DataFrame(race_input_layer)
print(testdf.shape)
no_nans = testdf.dropna()
print(no_nans.shape)

NameError: name 'stop' is not defined

In [ ]:
testdf

NameError: name 'testdf' is not defined

In [ ]:
len(dog_stats_df)

1020369

In [ ]:
with open("142k fasttrack FFNN new margins.npy", "wb") as fp:   #Pickling
    
    pickle.dump(full_details_new, fp)

In [ ]:
file = open(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\pytorch\New Model\DATA\total_list_new_box_index.npy", "rb")
data = pickle.load(file)
# seperate out classes from inputs
raceIDs, inputs, classes, prices, margins, betfairSP,names = zip(*data)
for i, race_id in enumerate(raceIDs):
    #print(i)
    if race_id =="140918784":
        first_arg = classes
        ins = inputs[i]
        cl = classes[i]
        dognames = names[i]
        print(i,race_id,cl, dognames)
        x_test = resultsdf[resultsdf['race_id']==race_id]
        # print(i,race_id)
        # print(prices[i])
        break
        

In [ ]:
print(full_details_new)

In [ ]:
raceIDs, inputs, classes, prices, margins, betfairSP = zip(*pickle.load(open('total_list_new_box_test.npy', 'rb')))

In [ ]:
for i, race_id in enumerate(raceIDs):
    if race_id =="140918784":
        first_arg = classes
        ins = inputs[i]
        cl = classes[i]
        print(i,race_id)
        x_test = resultsdf[resultsdf['race_id']==race_id]
        # print(i,race_id)
        # print(prices[i])
        break
        

In [ ]:
raceIDs[1285]

In [ ]:
cl

In [ ]:
resultsdf[resultsdf['race_id']=='128662384']

In [ ]:
dfx = pd.DataFrame(dog_stats_df, columns=['dogid','dog_name','raceid','race_grade','date','trackOHE','track_name','dist','stats','place','startprice','margin','bfSP', 'runtime'])
dfx.head(10)

,dogid,dog_name,raceid,race_grade,date,trackOHE,track_name,dist,stats,place,startprice,margin,bfSP,runtime
0,314368991,CALLUM ARI,498695480,Mixed 6/7 Final,2019-12-01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Healesville,350.0,"[5, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, ...",1.0,11.5,0.00,19.500000,19.39
1,314367487,RIO HUSTLER,498695480,Mixed 6/7 Final,2019-12-01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Healesville,350.0,"[2, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, ...",2.0,6.3,0.89,7.200000,19.45
2,291188717,THURSDAY THUNDER,498695480,Mixed 6/7 Final,2019-12-01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Healesville,350.0,"[8, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, ...",3.0,38.6,1.93,40.000000,19.52
3,296096016,ANNIHILATION,498695480,Mixed 6/7 Final,2019-12-01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Healesville,350.0,"[1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, ...",4.0,7.9,2.01,10.456086,19.53
4,282125654,GRUMPY DEREK,498695480,Mixed 6/7 Final,2019-12-01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Healesville,350.0,"[7, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, ...",5.0,5.4,3.33,7.543589,19.62
5,302937173,SUPERIOR DUNKLEY,498695480,Mixed 6/7 Final,2019-12-01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Healesville,350.0,"[3, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, ...",7.0,37.3,5.74,28.858711,19.79
6,214382420,SUPERIOR POWER,498695480,Mixed 6/7 Final,2019-12-01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Healesville,350.0,"[4, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, ...",8.0,1.9,6.87,2.280000,19.87
7,296096015,WORTHY PRINCE,498695480,Mixed 6/7 Final,2019-12-01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Healesville,350.0,"[6, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, ...",6.0,19.1,3.66,17.287920,19.64
8,328356413,CRAZY OLD BIRD,500539128,Maiden Final,2019-12-03,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Horsham,410.0,"[6, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, ...",8.0,14.6,19.74,11.000000,24.67
9,331308298,SILVER COMPASS,500539128,Maiden Final,2019-12-03,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Horsham,410.0,"[2, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, ...",7.0,40.6,15.69,110.000000,24.38


In [ ]:
betfair_df

,EVENT_ID,EVENT_DT,SELECTION_ID,BSP,dog,dateF,dog_name
2,138449114,31-12-2017 08:08,9117762,40.000000,Magic Moonlight,2017-12-31,MAGIC MOONLIGHT
3,138449293,31-12-2017 09:34,10880108,7.000000,Better Size,2017-12-31,BETTER SIZE
5,138449295,31-12-2017 09:54,11342999,6.800000,Midnight Dasher,2017-12-31,MIDNIGHT DASHER
34,138448884,31-12-2017 02:45,12190032,9.600000,Redda,2017-12-31,REDDA
37,138424801,31-12-2017 11:05,12600557,21.000000,Navajo Warrior,2017-12-31,NAVAJO WARRIOR
...,...,...,...,...,...,...,...
2921905,192881432,30-12-2021 23:37,42154306,5.218838,Graces Shout,2021-12-30,GRACES SHOUT
2921906,192851921,30-12-2021 05:32,42202110,10.500000,Tiggerlong Red,2021-12-30,TIGGERLONG RED
2921907,192848740,30-12-2021 01:06,42202125,147.227173,Lady Bella,2021-12-30,LADY BELLA
2921908,192848742,30-12-2021 01:23,42202133,15.096935,Clover Puzzle,2021-12-30,CLOVER PUZZLE


In [ ]:
test=datetime.strptime('2019-12-05', '%Y-%m-%d').date()

In [ ]:
test

datetime.date(2019, 12, 5)

In [ ]:
dfx[dfx['track_name'].str.contains("Christ")]

,dogid,dog_name,raceid,race_grade,date,trackOHE,track_name,dist,stats,place,startprice,margin,bfSP,runtime
4405,929490005,BROKEN PENNILESS,502563667,Class 3 heats,2019-12-05,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",Christchurch (NZ),295.0,"[4, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, ...",4.0,14.6,2.0,NaN,17.86
4406,133789054,CASH A ROO,502563667,Class 3 heats,2019-12-05,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",Christchurch (NZ),295.0,"[2, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, ...",5.0,6.3,2.1,NaN,17.86
4407,1994500013,FERAL KAIAKA,502563667,Class 3 heats,2019-12-05,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",Christchurch (NZ),295.0,"[8, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, ...",2.0,12.0,0.5,NaN,17.76
4408,230053844,INEFFABLE,502563667,Class 3 heats,2019-12-05,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",Christchurch (NZ),295.0,"[5, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, ...",6.0,10.3,2.6,NaN,17.89
4409,303248288,GOLDSTAR HALSEY,502563667,Class 3 heats,2019-12-05,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",Christchurch (NZ),295.0,"[7, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, ...",7.0,9.0,5.3,NaN,18.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1016262,792288915,HOMEBUSH SOCKS,846624041,Class 4,2022-11-25,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",Christchurch (NZ),295.0,"[2, 0.06101260321599304, 0.0015060769364373839...",3.0,12.6,3.0,NaN,17.58
1016263,379019065,TROUSSEAU,846624041,Class 4,2022-11-25,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",Christchurch (NZ),295.0,"[4, 0.05908664813493724, 0.0008120197245135202...",4.0,2.2,4.7,NaN,17.69
1016264,550411338,SAPPORO BALE,846624041,Class 4,2022-11-25,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",Christchurch (NZ),295.0,"[5, 0.06029199152542373, 0.0009832536469022316...",6.0,15.9,9.4,NaN,17.98
1016265,576143082,CAESAR LIES,846624041,Class 4,2022-11-25,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",Christchurch (NZ),295.0,"[6, 0.05958063787236349, 0.0009148839737714615...",7.0,3.1,10.7,NaN,18.06


In [ ]:
with open("dog_stats_df_FASTTRACK new margins.npy", "wb") as fp:   #Pickling
    
    pickle.dump(dfx, fp)

In [ ]:
file = open( "total_list.npy", 'rb')
data = pickle.load(file)
raceIDs, inputs, classes, prices, win_price, margins, betfairSP = zip(*data)
betfairSP

In [ ]:
bfSPdf = pd.DataFrame(betfairSP)
bfSPdf